## **El "Por Qué" del Seguimiento de Experimentos**

En ML, a menudo realizamos muchos experimentos para encontrar el mejor modelo para una tarea dada. Un experimento puede involucrar:

*   Probar diferentes arquitecturas de modelos (por ejemplo, RandomForest, GradientBoosting).
*   Ajustar hiperparámetros (por ejemplo, `max_depth`, `n_estimators`).
*   Usar diferentes conjuntos de características (features).
*   Variar las técnicas de preprocesamiento.

Llevar un registro de todos estos experimentos puede volverse rápidamente un proceso desordenado y propenso a errores. Aquí es donde entra en juego el seguimiento de experimentos.

## **¿Qué es el Experiment Tracking?**

El experiment tracking es el proceso de registrar y organizar de manera sistemática toda la información relacionada con tus experimentos de machine learning. Esto incluye:
*	Código: La versión exacta del código utilizada para ejecutar el experimento.
*	Datos: La versión del dataset empleado.
*	Parámetros: Los hiperparámetros y otras configuraciones.
*	Métricas: Las métricas de desempeño del modelo (por ejemplo, RMSE, accuracy).
*	Artefactos: El modelo entrenado, las visualizaciones y otros archivos generados.


## **¿Por qué es Importante?**

Reproducibilidad: Reproducir fácilmente resultados anteriores, lo cual es crucial para depuración (debugging) y para construir sobre trabajos previos.

* Organización: Mantener tus experimentos organizados y evitar perder el rastro de lo que ya has probado.
* Colaboración: Compartir tus resultados con colegas y colaborar de manera más efectiva.
* Comparación: Comparar fácilmente el desempeño de diferentes modelos y experimentos.
* Deployment: Hacer seguimiento de los modelos que se despliegan en producción y de su rendimiento.

## **Herramientas para Experiment Tracking**

Existen varias herramientas disponibles para experiment tracking, tanto de código abierto como comerciales. Algunas de las más populares incluyen:
* MLflow: Una plataforma open-source para el ciclo de vida de machine learning, incluyendo experiment tracking.
* Weights & Biases: Una plataforma comercial para experiment tracking y colaboración.
* Comet: Otra plataforma comercial con foco en experiment tracking y monitoreo de modelos.
* TensorBoard: Un kit de visualización para TensorFlow que también incluye funcionalidades de experiment tracking.

En este curso, nos enfocaremos en MLflow.

## **Training con MLFlow**

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle

## **Cargamos datos "procesados"**

In [2]:
df_jan = pd.read_parquet("data/processed/jan.parquet")
df_feb = pd.read_parquet("data/processed/feb.parquet")


## **Definir columnas a modelar**

In [3]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ["trip_distance"]

## Definir set de train y test

In [4]:
df_train = df_jan
df_val = df_feb

## **Preprocesar**

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

df_train = df_train.copy()
df_val   = df_val.copy()
df_train["categorical_dict"] = df_train[categorical].to_dict(orient="records")
df_val["categorical_dict"]   = df_val[categorical].to_dict(orient="records")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", DictVectorizer(), "categorical_dict"),
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median")),
                          ("scaler", StandardScaler())]), numerical),
    ]
)

X_train = preprocessor.fit_transform(df_train)
y_train = df_train["duration"].values
X_val   = preprocessor.transform(df_val)
y_val   = df_val["duration"].values

## **Entrenar**

Antes de ejecutar lo siguiente correr en la terminal en la misma carpeta donde esta el notebook:

`mlflow server --backend-store-uri sqlite:///backend.db`



In [6]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [7]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761786041422, experiment_id='0', last_update_time=1761786041422, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

experiment_name = "rf-nyc-taxi"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="random_forest_regressor"):
    rf = RandomForestRegressor(n_estimators=100, random_state=0, max_depth=10)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

🏃 View run random_forest_regressor at: http://127.0.0.1:5000/#/experiments/1/runs/80c6356d909f40b78fcd9b4087c4e95f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


## **Probando otro modelo**

In [13]:
import xgboost as xgb

with mlflow.start_run():
    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_reg.fit(X_train, y_train)
    y_pred = xgb_reg.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

🏃 View run unique-wasp-692 at: http://127.0.0.1:5000/#/experiments/1/runs/7ce60270a60a40ddb6b88c24704a8876
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Nota: si te da error, debes instalar libomp con bre en caso de tener mac, para windows reinstala xgboost y vuelve a añadirlo con uv add. 

## **Probando autologgin**

In [14]:
import mlflow

experiment_name = "rf-nyc-taxi-autolog"
mlflow.set_experiment(experiment_name)
mlflow.autolog()

with mlflow.start_run():
    rf = RandomForestRegressor(n_estimators=100, random_state=0, max_depth=10)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)


2025/10/29 20:25:07 INFO mlflow.tracking.fluent: Experiment with name 'rf-nyc-taxi-autolog' does not exist. Creating a new experiment.
2025/10/29 20:25:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/10/29 20:25:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/10/29 20:25:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run bustling-koi-484 at: http://127.0.0.1:5000/#/experiments/2/runs/c45cc027e96645cc85b0b5df58ea58e5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
